# Generate Insights from multiple data sources using Watson Studio

## 1. Setup
To prepare your environment, you need to install some packages.

### 1.1 Install the necessary packages

You need the latest versions of these packages:<br>
- ibm_db: This extension is the implementation of Python Database API.<br>
- ibm-cos-sdk: This package allows Python developers to write software that interacts with IBM Cloud Object Storage.<br><br>

** Install the ibm_db: **

In [ ]:
!pip install ibm_db

** Install the ibm-cos-sdk: **

In [ ]:
!pip install ibm-cos-sdk

### 1.2 Import packages and libraries

Import the packages and libraries that you'll use:

In [ ]:
import ibm_db
import ibm_boto3
from botocore.client import Config
import pandas as pd

## 2. Configuration

Add configurable items of the notebook below

### 2.1 Global Variables

Add global variables.

In [ ]:
frames=[]

### 2.2 Add the data from local system(csv file)

Insert pandas DataFrame in the following cell and name it as <i><b>df1</b></i>

In [ ]:
#Insert Manchester.csv credentials here.

In [ ]:
frames.append(df1)

### 2.3 Add the data from Db2 and Db2 Warehouse

Insert pandas DataFrame in the following cell and name it as <i><b>df2</b></i>

In [ ]:
#Insert Db2 pandas DataFrame here.

In [ ]:
df2.rename(columns={'TYPES_OF_WATCHES': 'TYPES OF WATCHES', 'ORDER_METHOD': 'ORDER METHOD', 'STORE_LOCATION': 'STORE LOCATION', 'UNITS_SOLD': 'UNITS SOLD', 'MANUFACTURING_COST_PER_UNIT____USD_': 'MANUFACTURING COST PER UNIT ($ USD)', 'MANUFACTURING_COST____USD_': 'MANUFACTURING COST ($ USD)', 'UNIT_SALE_PRICE____USD_': 'UNIT SALE PRICE ($ USD)', 'REVENUE_GENERATED____USD_': 'REVENUE GENERATED ($ USD)', 'GROSS_PROFIT____USD_': 'GROSS PROFIT ($ USD)'}, inplace=True)
df2

In [ ]:
frames.append(df2)



Insert pandas DataFrame in the following cell and name it as <i><b>df3</b></i>

In [ ]:
#Insert Db2 warehouse pandas DataFrame here.

In [ ]:
df3.rename(columns={'TYPES_OF_WATCHES': 'TYPES OF WATCHES', 'ORDER_METHOD': 'ORDER METHOD', 'STORE_LOCATION': 'STORE LOCATION', 'UNITS_SOLD': 'UNITS SOLD', 'MANUFACTURING_COST_PER_UNIT____USD_': 'MANUFACTURING COST PER UNIT ($ USD)', 'MANUFACTURING_COST____USD_': 'MANUFACTURING COST ($ USD)', 'UNIT_SALE_PRICE____USD_': 'UNIT SALE PRICE ($ USD)', 'REVENUE_GENERATED____USD_': 'REVENUE GENERATED ($ USD)', 'GROSS_PROFIT____USD_': 'GROSS PROFIT ($ USD)'}, inplace=True)
df3

In [ ]:
frames.append(df3)

### 2.4 Integrate the data from csv file, Db2 and Db2 Warehouse

In [ ]:
result = pd.concat(frames, ignore_index=True)
result

### 2.5 Configure to the Db2 Warehouse for visualisation

Insert the connection credentials as <i><b>credentials_1</b></i> in the following cell after 
removing the current contents in the cell. 

In [ ]:
#Insert Db2 warehouse connection credentials here.

In [ ]:
dsn_driver = "IBM DB2 ODBC DRIVER"
dsn_database = credentials_1['database'] 
dsn_hostname = credentials_1['host']
dsn_port = "50000"                
dsn_uid = credentials_1['username']      
dsn_pwd = credentials_1['password']

dsn = (
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE={0};"
    "HOSTNAME={1};"
    "PORT={2};"
    "PROTOCOL=TCPIP;"
    "UID={3};"
    "PWD={4};").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)

conn = ibm_db.connect(dsn, "", "")

## 3. Send the data to Db2 Warehouse

In [ ]:
create_statement= 'CREATE TABLE "ALL" (\n"YEAR" VARCHAR,\n "TYPES OF WATCHES" VARCHAR,\n  "ORDER METHOD" VARCHAR,\n  "STORE LOCATION" VARCHAR,\n  "REVENUE GENERATED ($ USD)" VARCHAR, \n  "MANUFACTURING COST ($ USD)" VARCHAR,\n "UNITS SOLD" VARCHAR ,\n  "MANUFACTURING COST PER UNIT ($ USD)" VARCHAR,\n  "GROSS PROFIT ($ USD)" VARCHAR,\n  "UNIT SALE PRICE ($ USD)" VARCHAR\n)'
create_statement=create_statement.replace('VARCHAR', 'VARCHAR(256)')
ibm_db.exec_immediate(conn, create_statement)

In [ ]:
for index, row in result.iterrows():
    print (row['YEAR'], row['TYPES OF WATCHES'], row['ORDER METHOD'], row['STORE LOCATION'], row['REVENUE GENERATED ($ USD)'], row['MANUFACTURING COST ($ USD)'], row['UNITS SOLD'], row['MANUFACTURING COST PER UNIT ($ USD)'], row['GROSS PROFIT ($ USD)'], row['UNIT SALE PRICE ($ USD)'])
    print("-----------------------------------")
    a="\'"
    w = a+row['TYPES OF WATCHES']+a
    o = a+row['ORDER METHOD']+a
    s = a+row['STORE LOCATION']+a
    insert = 'INSERT INTO DASH14176.ALL VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)' %(row['YEAR'], w, o, s, row['REVENUE GENERATED ($ USD)'], row['MANUFACTURING COST ($ USD)'], row['UNITS SOLD'], row['MANUFACTURING COST PER UNIT ($ USD)'], row['GROSS PROFIT ($ USD)'], row['UNIT SALE PRICE ($ USD)'])
    ibm_db.exec_immediate(conn, insert)